In [ ]:
# change into root directory to the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from neurolib.models.aln import ALNModel
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.evolution import Evolution
import neurolib.utils.functions as func

from neurolib.utils.loadData import Dataset
ds = Dataset("gw")

# sett dpi for notebooks on server
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['image.cmap'] = 'plasma'

In [ ]:
aln = ALNModel(Cmat = ds.Cmat, Dmat = ds.Dmat, simulateBOLD=True) # simulates the whole-brain model in 10s chunks by default if simulateBOLD == True
# Resting state fits
aln.params['mue_ext_mean'] = 1.57
aln.params['mui_ext_mean'] = 1.6
aln.params['sigma_ou'] = 0.09
aln.params['b'] = 5.0
aln.params['signalV'] = 2
aln.params['dt'] = 0.2
aln.params['duration'] = 0.2 * 60 * 1000 #ms

aln.run()

In [ ]:
def evaluateSimulation(traj):
    rid = traj.id
    #print("Running run id {}".format(rid))

    #model = evolution.loadIndividual(traj)
    model = evolution.getModelFromTraj(traj)  
    model.run()
    
    # -------- fitness evaluation here --------
    
    scores = []
    for i, fc in enumerate(ds.FCs):#range(len(ds.FCs)):
        fc_score = func.matrix_correlation(func.fc(model.BOLD.BOLD[:, 5:]), fc)
        scores.append(fc_score)
    
    fitness = np.mean(scores)
    
    fitness_tuple = ()
    fitness_tuple += (fitness,fitness ,)
    fitness_tuple = tuple(scores)
    
    print(fitness_tuple)
    return fitness_tuple, {}

In [ ]:
pars = ParameterSpace(['mue_ext_mean', 'mui_ext_mean'], [[0.0, 2.0], [0.0, 2.0]])

In [ ]:
evolution = Evolution(evaluateSimulation, pars, weightList = [1.0, 1.0, 1.0], model = aln, POP_INIT_SIZE=4, POP_SIZE = 4, NGEN=2)

In [ ]:
evolution.run(verbose = False)

# Analysis

In [ ]:
evolution.info()

In [ ]:
evolution.popHist

In [ ]:
evolution.dfPop

In [ ]:
# This will load results from disk in case the session is 
# started newly and the trajectory is not in memory
traj = evolution.loadResults()

In [ ]:
gens, all_scores = evolution.getScoresDuringEvolution(reverse=True)

In [ ]:
plt.figure(figsize=(8, 4), dpi=200)   
plt.plot(gens, np.nanmean(all_scores, axis=1))
plt.fill_between(gens, np.nanmin(all_scores, axis=1), np.nanmax(all_scores, axis=1), alpha=0.3)
plt.xlabel("Generation #")
plt.ylabel("Score")